<span style="font-size: 2em; font-weight:bold">AI 70's Country</span>

In [1]:
import warnings
warnings.filterwarnings('ignore')

import scipy
import numpy as np
import matplotlib
import pandas as pd
import statsmodels
import sklearn
import tensorflow
import keras

from keras.models import Sequential, load_model
from keras.layers import Dense,LSTM,Dropout
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import History
import string
import pyarrow as pa


import plotly.graph_objs as go
import plotly
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected = True)
import plotly.offline as offline
from plotly import tools

import json
import time

Using TensorFlow backend.


As always, data prep is the hardest part of the project.  Because I am going to use a validation set in training my model, and because Keras uses the last n% of the data as the validation set, I want to shuffle the lyrics so that my validation set contains a better representation of all the data - not just the last song.  I also want to get the most originality that I can out of the model, so I will eliminate duplicate lyrics (some songs have refrains that repeat multiple times).  

In [2]:
lyricsCSV = pd.read_csv('lyricsTrain_25.csv',encoding='ISO-8859-1')
lyricsCSV.sort_values(lyricsCSV.columns[0],inplace=True)
lyricsCSV.drop_duplicates(keep='first',inplace=True)
lyricsCSV = lyricsCSV.sample(frac=1)



lyricsCSV.to_csv('lyrics.txt',sep='\t',index=False)
l = open('lyrics.txt','r')
lyrics = l.read()
l.close()

Remove lines and print

In [3]:
tokens = lyrics.split()
lyrics = ' '.join(tokens)
print(lyrics)

I beg your pardon Jolene, Jolene There'll be a load of compromisin' And my soft shoes shining And shakin' me up so that all I really know And I wonder just how long Somehow you needed me Money made from selling a hog Come along and share the good times while we can When ever I chance to meet, some old friends on the street On the sunday morning sidewalk I pull out my fiddle and I rosin up the bow "He said ""Hello, boys"" and then he gave us a grin 'n' said" But myself I can't deceive I know it's only make believe My luck was so good I could do no wrong And papa, I sure hope you understand (La la la la la la la, when you're hot, you're hot) I'll just cry all night long But his pride won't let him do things to make you think he's right And it's so good to be back home again With flaming locks of auburn hair My only prayer will be some day you'll care for me but it's only make believe Well a lot of things have changed since a way back then Newberry's train songs and Blue Eyes Cryin' in th

One final thing, when I look at the above lyrics, I seem to see a LOT of quotation marks.  So, I am going to just replace those with a space.  

In [4]:
lyrics = lyrics.replace('"',' ')

Now, we can build sequences of characters that will be used to predict a final character

In [5]:
length = 100 # Length of the characer sequences (because we have so much verbage,
             # we can use a relatively large number)
sequences = list()
for i in range(length, len(lyrics)):
    seq = lyrics[i-length:i+1]
    sequences.append(seq)

Create and save a .txt file of our sequences with line endings

In [6]:
data = '\n'.join(sequences)
file = open('char_sequences.txt','w')
file.write(data)
file.close()

Create a dictionary of character:number mappings 

In [7]:
file = open('char_sequences.txt','r')
raw_text = file.read()
file.close()

lines = raw_text.split('\n')

chars = sorted(list(set(raw_text)))
mapping = dict((c, i) for i, c in enumerate(chars))

# Save the mapping as json 
json_map = json.dumps(mapping)
__ = open('mapping.json','w')
__.write(json_map)
__.close()

Use the dictionary to create sequences of numbers only (numbers that describe the characters)

In [8]:
sequences = list()
for line in lines:
    encoded_seq = [mapping[char] for char in line]
    sequences.append(encoded_seq)
    


Create input sets (with 99 characters) and output sets (1 character) and then one-hot code the sets so we can use them to train the model.

In [9]:
vocab_size = len(mapping)
sequences = np.array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
sequences = [to_categorical(x, num_classes=vocab_size) for x in X] #one-hot code input
X = np.array(sequences)
y = to_categorical(y, num_classes=vocab_size) #one-hot code output

Fit the model with tuning parameters determined by trial and error.

In [10]:

def countryTrain(paramUnits,paramEpochs,paramValSplit,paramShuffle,paramBatchSize,paramDropout):

    #Define callbacks
    es = keras.callbacks.EarlyStopping(monitor = 'acc',min_delta = .01, patience = 3, mode = 'max',verbose=1)
    mc = keras.callbacks.ModelCheckpoint('model.drop_{}.best'.format(str(paramDropout)), monitor='loss', mode='min', save_best_only=True) # Keep best model


    # define and fit model
    model = Sequential()
    model.add(LSTM(paramUnits, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dropout(paramDropout))
    model.add(Dense(vocab_size, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


    modelLyrics = model.fit(X, y, epochs = paramEpochs, validation_split = paramValSplit, 
                            shuffle = paramShuffle, batch_size = paramBatchSize, verbose=1,callbacks=[es,mc])

   
    history = pd.DataFrame(modelLyrics.history)
    history.to_csv('modelLyricsHistory_drop_{}.csv'.format(str(paramDropout)),index=False)



In [11]:
#Define parameters  -
units = 512  # From the mentioned article in data science
epochs = 100  # Just a large number since I am using early stopping
validationSplit = 0.1 # My data set is small so I want to use as much as possible to train vs. validate
shuffle = True
batchSize = 64 # Doubled the default batch size to speed up training
dropOut = .5  # http://papers.nips.cc/paper/4878-understanding-dropout.pdf


for i in range(0,6):
    d = i*0.1
    countryTrain(units,epochs,validationSplit,shuffle,batchSize,d)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 20118 samples, validate on 2236 samples
Epoch 1/100
20118/20118 [==============================] - 195s 10ms/step - loss: 2.8877 - acc: 0.2321 - val_loss: 2.5098 - val_acc: 0.3108
Epoch 2/100
20118/20118 [==============================] - 196s 10ms/step - loss: 2.3632 - acc: 0.3427 - val_loss: 2.2894 - val_acc: 0.3614
Epoch 3/100
20118/20118 [==============================] - 193s 10ms/step - loss: 2.1848 - acc: 0.3829 - val_loss: 2.1500 - val_acc: 0.3940
Epoch 4/100
20118/20118 [==============================] - 192s 10ms/step - loss: 2.0299 - acc: 0.4170 - val_loss: 2.0249 - val_acc: 0.4141
Epoch 5/100
20118/20118 [==============================] - 192s 10ms/step - loss: 1.8996 - acc: 0.4484 - val_loss: 1.9466 - val_acc: 0.4369
Epoch 6/100
20118/20118 [==============================] - 193s 10ms/step - loss: 1.7737 - acc: 0.4832 - val_loss: 1.8777 - val_acc

 2560/20118 [==>...........................] - ETA: 5:48 - loss: 3.3081 - acc: 0.1672

KeyboardInterrupt: 

In [ ]:
# store history

Create a function that encodes a kickoff text string and then plugs it into our trained model

# generate a sequence of characters with a language model
def generate_seq(model, mapping, seq_length, seed_lyric, n_chars):
    lyrics = seed_lyric
    for __ in range(n_chars):
    # encode the characters as integers
        encoded = [mapping[char] for char in lyrics]
    # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
    # one hot encode
        encoded = to_categorical(encoded, num_classes=len(mapping))
    # predict character
        yhat = model.predict_classes(encoded, verbose=0)
    # reverse map integer to character
        out_char = ''
        for char, index in mapping.items():
            if index == yhat:
                out_char = char
                break
    # append to input
        lyrics += char
    return lyrics

Is there a song other than Stairway to Heaven that I could have used for the kickoff sequence?

startLyrics = "As I walk through the valley of the shadow of death I take a look at my life and realize there's not"

Run the model and print the lyrics

model = load_model('model_5_31.best')
lyricsFinal = generate_seq(model,mapping,length,startLyrics,1000)
print(lyricsFinal)

df = pd.read_json('modelLyricsHistory.json')

df